# Rating Product & Sorting Reviews in Amazon

---------

## İş Problemi

E-ticaretteki en önemli problemlerden bir tanesi ürünlere satış
sonrası verilen puanların doğru şekilde hesaplanmasıdır. Bu
problemin çözümü e-ticaret sitesi için daha fazla müşteri
memnuniyeti sağlamak, satıcılar için ürünün öne çıkması ve satın
alanlar için sorunsuz bir alışverişdeneyimi demektir. Bir diğer
problem ise ürünlere verilen yorumların doğru bir şekilde
sıralanması olarak karşımıza çıkmaktadır. Yanıltıcı yorumların öne
çıkması ürünün satışını doğrudan etkileyeceğinden dolayı hem
maddi kayıp hem de müşteri kaybına neden olacaktır. Bu 2 temel
problemin çözümünde e-ticaret sitesi ve satıcılar satışlarını
arttırırken müşteriler ise satın alma yolculuğunu sorunsuz olarak
tamamlayacaktır.


## Veri Seti Hikayesi

Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra
buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri
ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average
Bidding uygulanmıştır.

*Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average Bidding uygulanmıştır.*

In [2]:
### Değişkenler

##### reviewerID : Kullanıcı ID’si
##### asin : Ürün ID’si
##### reviewerName : Kullanıcı Adı
##### helpful : Faydalı değerlendirme derecesi
##### reviewText : Değerlendirme
##### overall : Ürün rating’i
##### summary : Değerlendirme özeti
##### unixReviewTime : Değerlendirme zamanı
##### reviewTime : Değerlendirme zamanı Raw
##### day_diff : Değerlendirmeden itibaren geçen gün sayısı
##### helpful_yes : Değerlendirmenin faydalı bulunma sayısı
##### total_vote : Değerlendirmeye verilen oy sayısı

## To do List:

**GÖREV 1**:  Average Rating’i güncel yorumlara göre hesaplayınız ve var olan average rating ile kıyaslayınız.

**GÖREV 2**:  Ürün için ürün detay sayfasında görüntülenecek 20 review’i belirleyiniz.



----------

### Görev 1

In [11]:
# Pandas kütüphanesini import edelim;

import pandas as pd
import numpy as np
# pandas görüntü ayarlarını yapalım;

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [12]:
# veri setinin dahil edelim;

df_ = pd.read_csv(r"C:\Users\kkakt\Desktop\projects\Rating & Sorting Projects\dataset\amazon_review.csv")

df = df_.copy()

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.000,best deal around,1373673600,2013-07-13,513,0,0


In [27]:
# veri setinde ürünün ortalama puanını hesaplayalım;

averaga_rating = df["overall"].mean()


In [31]:
# time_based weighted rating hesaplaması yapalım;

# son yapılan yorumların ağırlığına göre istatiksel betimlmeyi inceleyelim;
df["day_diff"].describe([.1,.2,.3,.4,.5,.6,.7,.8,.9,.99]).T

count   4915.000
mean     437.367
std      209.440
min        1.000
10%      167.000
20%      248.000
30%      311.000
40%      361.000
50%      431.000
60%      497.400
70%      562.800
80%      638.000
90%      708.000
99%      943.000
max     1064.000
Name: day_diff, dtype: float64

In [55]:
# fonksiyonumuzu hazırlayalım;

def time_based_weighted_rating(df,w1,w2,w3,w4):
    return df.loc [df["day_diff"]<= 150,"overall"].mean() * w1/100 +\
        df.loc [(df["day_diff"] > 150) & (df["day_diff"]<=361),"overall"].mean() * w2/100 +\
            df.loc [(df["day_diff"]>361) & (df["day_diff"]<=600),"overall"].mean() * w3/100 +\
                    df.loc [df["day_diff"]>600,"overall"].mean() * w4/100
    pass

# zaman farkınına göre bir katsayı oluşturarak puan hesaplamasını yapalım;
time_based_rating = time_based_weighted_rating(df,36,25,20,19)
print(f"Zaman Ağırlıklı: {time_based_rating}\nÜrün Ortalama Puan: {averaga_rating}")

Zaman Ağırlıklı: 4.631115164949879
Ürün Ortalama Puan: 4.587589013224822


## Görev 2

In [62]:
# Olumsuz yorum sayısını bir değişkene atayalım;

df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

df[df["total_vote"]>10]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no
93,A837QPVOZ9YAD,B007WTAJTO,Airedad,"[15, 21]",I'm amazed. I ordered this from Amazon on Tue...,5.000,Very fast class 10 card - and excellent servic...,1343174400,2012-07-25,866,15,21,6
315,A2J26NNQX6WKAU,B007WTAJTO,"Amazon Customer ""johncrea""","[38, 48]",Bought this card to use with my Samsung Galaxy...,5.000,Samsung Galaxy Tab2 works with this card if re...,1344816000,2012-08-13,847,38,48,10
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,73
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.000,I own it,1393545600,2014-02-28,283,60,68,8
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,68
2751,A19R7GVV216QKY,B007WTAJTO,Kunchok,"[8, 118]","If price is also double of 64 GB card, then it...",5.000,Price??,1393286400,2014-02-25,286,8,118,110
2909,A10B6G6W3DW1EY,B007WTAJTO,Luopo,"[53, 236]",I know armed with this in my Android tablet an...,4.000,Win Win situation,1393200000,2014-02-24,287,53,236,183
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,77
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.000,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25,3
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,27


In [74]:
# score_average_rating fonksiyonunu tanımlayalım;
def score_pos_neg_diff(up,down):
    return (up - down)

def score_average_rating(up,down):
    if up == 0:
        return 0
    return (up / (up+down))

# wilson_lower_bound fonksiyonunu tanımlayalım;
def wilson_lower_bound(up, down, confidence=0.95):
    import math
    import scipy.stats as st 
    """
    Wilson Lower Bound Score hesapla

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)


In [75]:
# veri seti içinde çeşitli skorları oluşturalım;

df["score_pos_neg_diff"] = df.apply(lambda x: score_pos_neg_diff(x["helpful_yes"], x["helpful_no"]),axis=1)
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]),axis=1)
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]),axis=1)



In [77]:
# wilson_lower_bound skoruna göre ilk 20 yorumu sıralayalım;

df.sort_values("wilson_lower_bound",ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,68,1884,0.966,0.958
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,77,1351,0.949,0.937
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,126,1442,0.926,0.912
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,73,349,0.853,0.819
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,4,41,0.918,0.808
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.000,I own it,1393545600,2014-02-28,283,60,68,8,52,0.882,0.785
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,27,85,0.806,0.732
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.000,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25,3,19,0.880,0.700
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.000,Awesome Card!,1339200000,2012-09-06,823,51,65,14,37,0.785,0.670
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.000,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,27,55,0.752,0.664
